# 1. 고급 구조 패턴

## 1. Batch Normalization

- 이전 예제는 모델에 데이터를 주입하기 전에 정규화했다. 


- 하지만 데이터 정규화는 네트워크에서 일어나는 모든 변환 후에도 고려되어야 한다. Dense나 Conv2D 층에 들어가는 데이터의 평균은 0이고 분산이 1이더라도 출력되는 데이터가 동일한 분포를 가질 것이라고 기대하기는 어렵다.


- 배치 정규화는 훈련하는 동안 평균과 분산이 바뀌더라도 이에 적응하여 데이터를 정규화한다. 훈련 과정에서 사용된 배치 데이터의 평균과 분산에 대한 지수 이동 평균을 내부에 유지합니다. 배치 정규화의 주요 효과는 잔차 연결과 매우 흡사하게 그래디언트의 전파를 돕는 것이다. (입력에 비하여 활성화 함수의 출력이 너무 작거나 커지면 변화율이 급격히 작아져 역전파되는 그래디언트도 매우 줄어들게 된다. 배치 정규화는 입력과 출력의 분포를 유지하도록 그래디언트가 더 잘 전파된다.)

In [2]:
# BatchNormalization은 일반적으로 합성곱이나 완전 연결 층 다음에 사용한다.
# data_format을 channel_lats로 사용하는 경우에 특성 축이 -1이다. (기본)
# conv_model.add(layers.Conv2D(32, 3, activation='relu'))
# conv_model.add(layers.BatchNormalization())

# data_format을 channel_first로 사용하는 경우에 특성 축이 1이다. (0차원: 배치)
# dense_model.add(layers.Dense(32, activation='relu'))
# dense_model.add(layers.BatchNormalization())

## 2. 깊이별 분리 합성곱

- SeperableConv2D Conv2D를 대체하면서 더 가볍고 더 빠르다. (depthwise seperable convloution)

- 이 층은 입력 채널별 따로따로 공간 방향의 합성곱을 수행합니다. 그 다음 점별 합성곱을 통해 출력 채널을 합칩니다. 이는 공간 특성의 학습과 채널 방향 특성의 학습을 분리하는 효과를 냅니다. 입력에서 공간상 위치는 상관관계가 크지만 채널별로는 매우 독립적이라고 가정하면 타당합니다. 이는 모델 파라미터와 연산 수를 크게 줄여주기 떄문에 작고 더 빠른 모델을 만듭니다. 

- 합성곱을 통해 더 효율적으로 표현을 학습하기 때문에 적은 데이터로도 더 좋은 표현을 학습하고 결국 성능이 더 좋은 모델을 만든다. <strong>제한된 데이터로 작은 모델을 처음부터 훈련시킬 떄 특히 더 중요하다</strong>

In [1]:
import keras
from keras.models import Sequential, Model
from keras.utils import to_categorical
from keras import layers

Using TensorFlow backend.


In [2]:
import numpy as np

In [3]:
x_train = np.random.random((1000, 64, 64, 3))
y_train = np.random.randint(1, 10, size=1000)

In [4]:
y_train = to_categorical(y_train)

In [5]:
x_train.shape, y_train.shape

((1000, 64, 64, 3), (1000, 10))

In [6]:
height = 64
width = 64
channels = 3
num_classes = 10

In [7]:
model = Sequential()
model.add(layers.SeparableConv2D(32, 3, activation='relu', input_shape=(height, width, channels,), name='first_conv'))
model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(4, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

model.summary()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
first_conv (SeparableConv2D) (None, 62, 62, 32)        155       
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 60, 60, 64)        2400      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
separable_conv2d_2 (Separabl (None, 28, 28, 64)        4736      
_________________________________________________________________
separable_conv2d_3 (Separabl (None, 26, 26, 128)       8896      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 128)       0         
_________________________________________________________________
separable_conv2d_4 (Separabl (None, 11, 11, 4)         1668      
__________

In [8]:
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='log_depthwise/',
        histogram_freq=1, # 1 에포크마다 활성화 출력 히스토그램 기록
    )
]
history = model.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.2, callbacks=callbacks)

Train on 800 samples, validate on 200 samples
Epoch 1/20
800/800 [==============================] - 5s 6ms/step - loss: 2.2997 - acc: 0.1113 - val_loss: 2.2949 - val_acc: 0.1100
Epoch 2/20
800/800 [==============================] - 4s 5ms/step - loss: 2.2931 - acc: 0.1113 - val_loss: 2.2895 - val_acc: 0.1100
Epoch 3/20
800/800 [==============================] - 4s 5ms/step - loss: 2.2874 - acc: 0.1113 - val_loss: 2.2841 - val_acc: 0.1100
Epoch 4/20
800/800 [==============================] - 4s 5ms/step - loss: 2.2815 - acc: 0.1113 - val_loss: 2.2784 - val_acc: 0.1100
Epoch 5/20
800/800 [==============================] - 4s 5ms/step - loss: 2.2754 - acc: 0.1113 - val_loss: 2.2732 - val_acc: 0.1100
Epoch 6/20
800/800 [==============================] - 4s 5ms/step - loss: 2.2695 - acc: 0.1113 - val_loss: 2.2679 - val_acc: 0.1100
Epoch 7/20
800/800 [==============================] - 4s 5ms/step - loss: 2.2632 - acc: 0.1113 - val_loss: 2.2620 - val_acc: 0.1100
Epoch 8/20
800/800 [==========

## 3. 하이퍼파라미터 최적화

전형적인 하이퍼파라미터 최적화 과정은 다음과 같습니다.

1. 일련의 하이퍼파라미터를 선택합니다.
2. 선택된 하이퍼파라미터로 모델을 만듭니다.
3. 훈련 데이터에 학습하고 검증 데이터에 최종 성능을 측정합니다.
4. 다음으로 시도할 하이퍼파라미터를 선택합니다.
5. 이 과정을 반복합니다.
6. 마지막으로 테스트 데이터에서 성능을 측정합니다.

- bayesian optimization, genetic algorithm, random search

- 가장 단순하지만 종종 랜덤 탐색이 제일 좋은 방법일 때가 많습니다. 대규모 자동화하여 하이퍼파라미터 최적화를 진행할 떄 기억해야 할 것은 검증 세트의 과적합이다. 이 점을 항상 기억하세요.

## 4. 모델 앙상블

- 앙상블은 독립적으로 훈련된 다른 종류의 모델이 각기 다른 장점을 가지고 있다는 가정을 바탕으로 한다.

- 앙상블이 잘 작동하게 만드는 핵심은 분류기의 다양성입니다.

- 그렇기 떄문에 가능한 최대한 다르면서 좋은 모델을 앙상블해야 한다. 일반적으로 매우 다른 구조를 가지거나 다른 종류의 머신 러닝 방법을 말한다. 같은 네트워크를 랜덤 초기화를 다르게 하여 따로 따로 여러 번 훈련해서 앙상블하는 것은 거의 해볼 가치가 없다.

- 모든 문제에 적용하지는 못하지만 실전에서 잘 동작하는 한 가지 방법은 트리 기반 모델이나 심층 신경망을 앙상블하는 것입니다. 특별하게 모델 중 하나는 다른 방법을 사용해서 만들었습니다.(RGF 모델) 이 모델은 다른 모델보다 눈에 띄게 나빴습니다. 당연히 앙상블에 낮은 가중치를 할당했습니다. 하지만 놀랍게도 이 모델이 전체 앙상블의 성능을 크게 향상시켰습니다.

- 최상의 모델이 얼마나 좋은지보다 앙상블의 후보 모델이 얼마나 다양한지가 중요하다. 최근에 실전에서 매우 성공적으로 사용되는 기본 앙상블 스타일은 딥러닝과 얕은 모델을 섞은 넓고 깊은 모델입니다. 이런 모델은 심층 신경망과 많은 선형 모델을 함께 훈련합니다. 다양한 종류의 모델을 함께 훈련하는 것은 모델 앙상블을 만드는 또 다른 방법입니다.

## 5. 정리

1. 미래에는 Depthwise Convolution이 일반적인 합성곱을 완전히 대체할 것입니다.

2. 심층 네트워크를 만들 떄 많은 하이퍼파라미터와 네트워크 구조를 선택해야 한다. 이 값들이 모여서 모델의 성능을 결정한다. 이런 선택은 랜덤 선택에 의존하지말고 하이퍼파라미터 공간을 조직적으로 탐색하는 것이 좋습니다.

3. 머신러닝 경연대회에서 우승하거나 어떤 문제에서 최상의 결과를 얻으려면 대규모로 모델을 앙상블하게 됩니다. 최적화가 잘된 가중 평균으로 만든 앙상블은 충분히 좋은 결과를 만듭니다. 다양성이 중요합니다.